```{contents}
```
## Chain of Thought

**Chain of Thought (CoT)** refers to the internal multi-step reasoning process an LLM uses to transform inputs into outputs.
In LangGraph, CoT is not just a prompt technique — it becomes a **first-class execution pattern** implemented through **stateful graphs, reasoning loops, and control flow**.

---

### **1. Conceptual Intuition**

A single LLM call is insufficient for complex tasks.
LangGraph makes reasoning **explicit, inspectable, iterative, and controllable**.

**Traditional LLM**

```
Input → LLM → Output
```

**LangGraph with CoT**

```
Input → Reason → Act → Observe → Reflect → (Repeat) → Answer
```

Each step is represented by **nodes** and **edges** in the graph.

---

### **2. Why Chain of Thought Matters**

| Capability                 | Without CoT | With CoT |
| -------------------------- | ----------- | -------- |
| Multi-step problem solving | Weak        | Strong   |
| Tool use                   | Fragile     | Robust   |
| Error recovery             | None        | Built-in |
| Complex planning           | Hard        | Natural  |
| Interpretability           | Low         | High     |

---

### **3. Chain of Thought as a Graph Pattern**

In LangGraph, CoT is implemented using:

* **State** → stores reasoning trace
* **Nodes** → perform each reasoning step
* **Edges** → control progression
* **Cycles** → allow refinement

```
[Reason] → [Tool] → [Observe] → [Reflect]
     ↑                         ↓
     └────────── Loop ─────────┘
```

---

### **4. State Schema for CoT**

```python
class State(TypedDict):
    question: str
    thoughts: list[str]
    observation: str
    answer: str
    done: bool
```

The **thoughts list** stores the evolving chain of reasoning.

---

### **5. Implementing CoT in LangGraph**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    question: str
    thoughts: list[str]
    answer: str
    done: bool

def reason(state):
    thought = f"Analyzing: {state['question']}"
    return {"thoughts": state["thoughts"] + [thought]}

def conclude(state):
    return {"answer": "Final Answer", "done": True}

def router(state):
    return END if state["done"] else "reason"

builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("conclude", conclude)

builder.set_entry_point("reason")
builder.add_conditional_edges("reason", lambda s: "conclude", {"conclude": "conclude"})
builder.add_conditional_edges("conclude", router, {"reason": "reason", END: END})

graph = builder.compile()
```

---

### **6. Variants of Chain of Thought in LangGraph**

| Variant           | Purpose                   |
| ----------------- | ------------------------- |
| Basic CoT         | Linear reasoning          |
| ReAct             | Reason + Act + Observe    |
| Self-Reflection   | Critique and improve      |
| Tree of Thoughts  | Branching reasoning paths |
| Program-aided CoT | LLM + code execution      |
| Debate CoT        | Multiple agents reason    |

---

### **7. Production Controls**

| Control         | Purpose                |
| --------------- | ---------------------- |
| Recursion limit | Prevent infinite loops |
| Checkpointing   | Resume execution       |
| Human review    | Oversight              |
| Memory store    | Long-term reasoning    |
| Logging         | Traceability           |
| Tool safety     | Prevent misuse         |

```python
graph.invoke(input, config={"recursion_limit": 10})
```

---

### **8. Best Practices**

* Store reasoning in structured state, not raw text
* Keep reasoning modular (one step per node)
* Use reflection loops for accuracy
* Enforce termination conditions
* Combine CoT with tools for reliability

---

### **9. Mental Model**

LangGraph transforms Chain of Thought from a **prompt trick** into an **explicit reasoning system**:

> **State + Control Flow + Iteration = Reliable Intelligence**


### Demonstration

In [2]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Define the State
# ---------------------------
class State(TypedDict):
    question: str
    thoughts: List[str]
    step: int
    answer: str
    done: bool

# ---------------------------
# 2. Reasoning Node
# ---------------------------
def reason(state: State) -> dict:
    new_thought = f"Step {state['step']}: Analyzing question"
    return {
        "thoughts": state["thoughts"] + [new_thought],
        "step": state["step"] + 1
    }

# ---------------------------
# 3. Reflection / Termination Check
# ---------------------------
def reflect(state: State) -> dict:
    if state["step"] >= 3:
        return {"answer": "Reasoning complete. Final answer produced.", "done": True}
    return {"done": False}

# ---------------------------
# 4. Router
# ---------------------------
def router(state: State):
    return END if state["done"] else "reason"

# ---------------------------
# 5. Build the Graph
# ---------------------------
builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("reflect", reflect)

builder.set_entry_point("reason")
builder.add_edge("reason", "reflect")
builder.add_conditional_edges("reflect", router, {"reason": "reason", END: END})

graph = builder.compile()

# ---------------------------
# 6. Execute
# ---------------------------
result = graph.invoke({
    "question": "Explain gravity simply",
    "thoughts": [],
    "step": 0,
    "answer": "",
    "done": False
}, config={"recursion_limit": 10})

result


{'question': 'Explain gravity simply',
 'thoughts': ['Step 0: Analyzing question',
  'Step 1: Analyzing question',
  'Step 2: Analyzing question'],
 'step': 3,
 'answer': 'Reasoning complete. Final answer produced.',
 'done': True}